In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

#ignore python warnings
warnings.filterwarnings('ignore')

In [2]:
#Read data from excel worksheets
df_Kayaking =pd.read_excel("V02Max_Combined.xlsx",sheet_name="Kayaking")
df_Athletes=pd.read_excel("V02Max_Combined.xlsx",sheet_name="Athletes")
df_Triathlon=pd.read_excel("V02Max_Combined.xlsx",sheet_name="Triathlon")
df_Fencing=pd.read_excel("V02Max_Combined.xlsx",sheet_name="Fencing")

In [3]:
#concatenate all sports data to form a single dataframe
df_data = pd.concat([df_Fencing,df_Kayaking,df_Triathlon],ignore_index=True)

Q1.Can we predict VO2max based on Age and Ethnicity:
 -VO2Max=A higher VO₂max is generally associated with a greater 
capacity for sustained exercise and is a predictor of endurance performance
in activities like running, cycling, and swimming.   

In [110]:

df_data_copy=df_data.copy()
df_Sum_VO2=df_data_copy.groupby('ID')['VO2'].sum().reset_index()

df_merge_VO2=pd.merge(df_Athletes, df_Sum_VO2[['ID', 'VO2']], on='ID')

df_merge_VO2['VO2max']=df_merge_VO2['VO2']/df_merge_VO2['Weight']


threshold=df_merge_VO2['VO2max'].median()

df_merge_VO2['VO2max_Binary'] = df_merge_VO2['VO2max'].apply(lambda x: 1 if x > threshold else 0)

gender_mapping = {'Male': 0, 'Female': 1}
ethnicity_mapping = {'African': 0, 'Asian': 1, 'White': 2, 'Latino': 3}  # Adjust as needed


df_merge_VO2['Gender_binary'] = df_merge_VO2['Gender'].map(gender_mapping)
df_merge_VO2['Ethnicity_numeric'] = df_merge_VO2['Ethnicity'].map(ethnicity_mapping)



In [50]:
df_merge_VO2

,ID,age,Weight,Height,P_vt1,P_vt2,sport,Gender,Ethnicity,First Name,Last Name,VO2,VO2max,VO2max_Binary,Gender_binary,Ethnicity_numeric
0,1,15,53.7,161,50,140,fencing,Female,African,Olivia,Martinez,2298.775092,42.807730,0,1,0
1,2,15,56.2,162,80,127,fencing,Male,African,Ethan,Reynolds,2171.152256,38.632602,0,0,0
2,3,16,89.9,191,155,300,fencing,Male,White,Jackson,Harper,6266.538536,69.705657,1,0,2
3,4,16,76.5,180,215,275,fencing,Male,White,Liam,Sanchez,5889.725663,76.989878,1,0,2
4,5,17,76.4,186,110,172,fencing,Female,White,Ava,Thompson,6627.133058,86.742579,1,1,2
5,6,17,65.6,169,131,186,fencing,Female,White,Sophia,Rodriguez,4702.172897,71.679465,1,1,2
6,7,15,49.8,165,65,108,fencing,Female,White,Grace,Simmons,1912.518617,38.403988,0,1,2
7,8,15,51.2,165,80,133,fencing,Female,White,Aiden,Bennett,2294.891090,44.822092,0,1,2
8,9,14,57.1,176,50,125,fencing,Female,African,Chloe,Price,2837.718226,49.697342,0,1,0
9,10,14,58.8,177,101,162,fencing,Female,African,Lily,Jordan,3241.183654,55.122171,0,1,0


In [101]:
X = df_merge_VO2[['age','Ethnicity_numeric']].values
y = df_merge_VO2.iloc[:,-1].values
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.3)
X_train.shape, X_test.shape
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()

X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X_train_std, y_train)
log_reg.score(X_test_std, y_test)


0.5